In [1]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.simplefilter("ignore")
from IPython.core.display import HTML, display

train = pd.read_csv('train_spam.csv', encoding='utf8')
train

,text_type,text
0,ham,make sure alex knows his birthday is over in f...
1,ham,a resume for john lavorato thanks vince i will...
2,spam,plzz visit my website moviesgodml to get all m...
3,spam,urgent your mobile number has been awarded wit...
4,ham,overview of hr associates analyst project per ...
...,...,...
16273,spam,if you are interested in binary options tradin...
16274,spam,dirty pictureblyk on aircel thanks you for bei...
16275,ham,or you could do this g on mon 1635465 sep 1635...
16276,ham,insta reels par 80 गंद bhara pada hai 👀 kuch b...


In [2]:
from sklearn.model_selection import train_test_split

train["text_type"] = train["text_type"].apply(lambda x: 1 if x=='spam' else 0)
X, y = train["text"], train["text_type"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((11394,), (4884,), (11394,), (4884,))

In [3]:
y.value_counts()

text_type
0    11469
1     4809
Name: count, dtype: int64

In [4]:
y_test.value_counts()

text_type
0    3475
1    1409
Name: count, dtype: int64

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)
X_train_vect.shape, X_test_vect.shape

((11394, 43830), (4884, 43830))

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

clf = LogisticRegression(random_state=0)
clf.fit(X_train_vect, y_train)
predicted = clf.predict(X_test_vect)

print('Train accuracy:', accuracy_score(y_train, clf.predict(X_train_vect)), 
      'Test accuracy:', accuracy_score(y_test, predicted))

Train accuracy: 0.9942074776197999 Test accuracy: 0.9443079443079443


In [7]:
y_val_proba = clf.predict_proba(X_test_vect)[:, 1]  
token2id = {token: i for i, token in enumerate(vectorizer.get_feature_names_out())}
importance = clf.coef_[0]
min_importance = importance.min()
max_importance = importance.max()
TOKEN_PATTERN = "(?:\w|\')+"
ids = [6, 13, 23, 34, 35, 40]
for i in ids:
    review_body = X_test.iloc[i]
    print(f'spam: {bool(y_test.values[i])}')
    review_tokens = re.findall(TOKEN_PATTERN, review_body.lower())
    html_string = '''
    <p style="font-size:16px; color:#000000; border: 2px solid #000; text-align: justify; background-color:#ffffff; border-radius: 25px; padding: 20px;">
    '''
    for token in review_tokens:
        if token in token2id:
            weight = importance[token2id[token]]
            if weight < 0:
                component = hex(int(255 - 255 * weight / min_importance))[2:]
                color = f'{component}{component}ff'
            else:
                component = hex(int(255 - 255 * weight / max_importance))[2:]
                color = f'ff{component}{component}'
        else:
            weight = 0.0
            color = 'ffffff'
        html_string += f'<span style="background-color: #{color}"; title="{weight:.2f}">{token}</span> '
    html_string += '</p>'
    display(HTML(html_string))

spam: True


spam: True


spam: False


spam: False


spam: False


spam: False
